![image](dependencies/open-science-prize.png)

## Higher Fidelity Graph States
In this notebook, we will prepare quantum circuits for a 7-qubit graph state and estimate the fidelity through stabilizer measurements using Qiskit. In this example, we use the CTMP method of error mitigation [1] and repeat the graph state measurement 16 times to find statistical error bars.

**To Do:
Modify the graph state preparation circuit or use your own methods of error mitigation to improve the graph state fidelity.**


[1] S. Bravyi, S. Sheldon, A. Kandala, D.C. McKay, J.M. Gambetta, Mitigating measurement errors in multi-qubit experiments, [arXiv:2006.14044](https://arxiv.org/abs/2006.14044) (2020).

## Imports

Begin by importing the necessary packages and defining the functions we will need for the stabilizer measurements.

In [1]:
### install Qiskit and other modules if you don't have them already
!pip install -r dependencies/requirements.txt --quiet

In [2]:
# Qiskit module
from qiskit import QuantumCircuit
import qiskit.circuit.library as circuit_library
import qiskit.quantum_info as qi
import qiskit.ignis.mitigation as mit

# Qiskit tools for noisy simulation
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.utils import insert_noise

# Qiskit tools for running and monitoring jobs
from qiskit import execute
from qiskit.tools.monitor import job_monitor

# Other imports
import numpy as np

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In order to run the circuits, first load the backend `ibmq_casablanca` from your account using the `IBMQ` provider. You will receive access to `ibm-q-community/ibmquantumawards/open-science` after registering for the Open Science Prize.

In [4]:
# Load IBMQ Account data
from qiskit import IBMQ
IBMQ.load_account()

# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-research', group='Tharrmashastha-S', project='main')
backend = provider.get_backend('ibmq_casablanca')
properties = backend.properties()

ibmqfactory.load_account:WARNING:2021-01-18 15:47:18,641: Credentials are already in use. The existing account in the session will be replaced.


## Preparing graph states

Here, we prepare the graph state circuit for 7-qubits using the `GraphState` function in Qiskit's circuit library. We define a graph that uses the connectivity map of the quantum system `ibmq_casablanca`.

In [5]:
num_qubits = 7

# adjacency matrix for `ibmq_casablanca`
adjmat = [
    [0, 1, 0, 0, 0, 0, 0], 
    [1, 0, 1, 1, 0, 0, 0], 
    [0, 1, 0, 0, 0, 0, 0], 
    [0, 1, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 1, 1, 0, 1], 
    [0, 0, 0, 0, 0, 1, 0]]

### Your code goes here

How would you prepare a graph state with high fidelity? In the example below, we create it using Qiskit's circuit library at the gate level. You may explore other methods for creating the graph states, including by using pulse-level techniques or accounting for the errors in the quantum system.

In [6]:
def create_graph_state():
    
    ### YOUR CODE GOES HERE -- START
    
    graph_state_circuit = circuit_library.GraphState(adjmat)
    
    ### YOUR CODE GOES HERE -- END
    
    return graph_state_circuit

In [7]:
# the graph state can be created using Qiskit's circuit library
state_circuit = create_graph_state()
state_circuit.draw()

┌───┐                  
q_0: ┤ H ├─■────────────────
     ├───┤ │                
q_1: ┤ H ├─■──■──■──────────
     ├───┤    │  │          
q_2: ┤ H ├────■──┼──────────
     ├───┤       │          
q_3: ┤ H ├───────■──■───────
     ├───┤          │       
q_4: ┤ H ├──────────┼──■────
     ├───┤          │  │    
q_5: ┤ H ├──────────■──■──■─
     ├───┤                │ 
q_6: ┤ H ├────────────────■─
     └───┘

## Creating and measuring stabilizers

We begin by defining functions to create stabilizer measurement circuits, and then appending them onto the circuit used to create the graph states.

[added]

An operator $A$ is called stabilizer of a state $|\psi\rangle$ if $A$ is such that $A|\psi\rangle = |\psi\rangle$. Essentially, $|\psi\rangle$ is an eigenvector of $A$ with the eigenvalue $1$.

We use stabilizer measurements here because of the fact that any given state can be uniquely represented using a set of stabilizers.

In [8]:
def compute_stabilizer_group(circuit):
    """Compute the stabilizer group for stabilizer circuit."""
    state = qi.Statevector.from_instruction(circuit)
    labels = []
    for i in qi.pauli_basis(state.num_qubits):
        val = round(qi.state_fidelity(i.to_matrix()[0], state, validate=False))
        if val != 0:
            label = i.to_labels()[0]
            if val == 1:
                label = '+' + label
            else:
                label = '-' + label
            labels.append(label)
    return labels

def stabilizer_coeff_pauli(stabilizer):
    """Return the 1 or -1 coeff and Pauli label."""
    coeff = 1
    pauli = coeff
    if stabilizer[0] == '-':
        coeff = -1
    if stabilizer[0] in ['+', '-']:
        pauli = stabilizer[1:]
    else:
        pauli = stabilizer
    return coeff, pauli

def stabilizer_measure_circuit(stabilizer, initial_circuit=None):
    """Return a stabilizer measurement circuits.
    
    Args:
        stabilizer (str): a stabilizer string
        initial_circuit (QuantumCircuit): Optional, the initial circuit.
    
    Returns:
        QuantumCircuit: the circuit with stabilizer measurements.
    """
    _, pauli = stabilizer_coeff_pauli(stabilizer)
    if initial_circuit is None:
        circ = QuantumCircuit(len(pauli))
    else:
        circ = initial_circuit.copy()
    for i, s in enumerate(reversed(pauli)):
        if s == 'X':
            circ.h(i)
        if s == 'Y':
            circ.sdg(i)
            circ.h(i)
    circ.measure_all()
    return circ

In [9]:
## Compute the stabilizers for this graph state
generators = qi.Clifford(state_circuit).stabilizer.pauli.to_labels()
stabilizers = compute_stabilizer_group(state_circuit)
print('Stabilizers:', stabilizers)
print('Generators:', generators)

Stabilizers: ['+IIIIIII', '+IIIIIZX', '+IIIIXIX', '+IIIIXZI', '-IIIZYXY', '+IIIZYYZ', '+IIIZZXZ', '+IIIZZYY', '+IIXXIIX', '+IIXXIZI', '+IIXXXII', '+IIXXXZX', '-IIXYYXZ', '-IIXYYYY', '-IIXYZXY', '+IIXYZYZ', '+IZIXIIX', '+IZIXIZI', '+IZIXXII', '+IZIXXZX', '-IZIYYXZ', '-IZIYYYY', '-IZIYZXY', '+IZIYZYZ', '+IZXIIII', '+IZXIIZX', '+IZXIXIX', '+IZXIXZI', '-IZXZYXY', '+IZXZYYZ', '+IZXZZXZ', '+IZXZZYY', '+XIIXIIX', '+XIIXIZI', '+XIIXXII', '+XIIXXZX', '-XIIYYXZ', '-XIIYYYY', '-XIIYZXY', '+XIIYZYZ', '+XIXIIII', '+XIXIIZX', '+XIXIXIX', '+XIXIXZI', '-XIXZYXY', '+XIXZYYZ', '+XIXZZXZ', '+XIXZZYY', '+XZIIIII', '+XZIIIZX', '+XZIIXIX', '+XZIIXZI', '-XZIZYXY', '+XZIZYYZ', '+XZIZZXZ', '+XZIZZYY', '+XZXXIIX', '+XZXXIZI', '+XZXXXII', '+XZXXXZX', '-XZXYYXZ', '-XZXYYYY', '-XZXYZXY', '+XZXYZYZ', '+YXYIYXY', '-YXYIYYZ', '-YXYIZXZ', '-YXYIZYY', '-YXYZIII', '-YXYZIZX', '-YXYZXIX', '-YXYZXZI', '-YXZXYXZ', '-YXZXYYY', '-YXZXZXY', '+YXZXZYZ', '-YXZYIIX', '-YXZYIZI', '-YXZYXII', '-YXZYXZX', '-YYYXYXZ', '-YYYXYYY', '-

In [58]:
## Append the stabilizer measurements to the graph state circuit 
stabilizer_circuits = [stabilizer_measure_circuit(stab, state_circuit)
                       for stab in stabilizers]

stabilizer_circuits[0].draw()

┌───┐                   ░ ┌─┐                  
   q_0: ┤ H ├─■─────────────────░─┤M├──────────────────
        ├───┤ │                 ░ └╥┘┌─┐               
   q_1: ┤ H ├─■──■──■───────────░──╫─┤M├───────────────
        ├───┤    │  │           ░  ║ └╥┘┌─┐            
   q_2: ┤ H ├────■──┼───────────░──╫──╫─┤M├────────────
        ├───┤       │           ░  ║  ║ └╥┘┌─┐         
   q_3: ┤ H ├───────■──■────────░──╫──╫──╫─┤M├─────────
        ├───┤          │        ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ┤ H ├──────────┼──■─────░──╫──╫──╫──╫─┤M├──────
        ├───┤          │  │     ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ┤ H ├──────────■──■──■──░──╫──╫──╫──╫──╫─┤M├───
        ├───┤                │  ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   q_6: ┤ H ├────────────────■──░──╫──╫──╫──╫──╫──╫─┤M├
        └───┘                   ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/═══════════════════════════╩══╩══╩══╩══╩══╩══╩═
                                   0  1  2  3  4  5  6

## Measurement mitigation

Noisy measurements impact our ability to accurately measure the state fidelity. For our default example we calibrate our measurements for the CTMP method using states with two-qubit excitations.

In [16]:
labels = ['0000000', '0000011', '0000101', 
          '0001001', '0001010', '0001100', 
          '0010001', '0010010', '0010100', '0011000', 
          '0100001', '0100010', '0100100', '0101000', '0110000', 
          '1000001', '1000010', '1000100', '1001000', '1010000', '1100000', 
          '1111111']
meas_cal_circuits, metadata = mit.expval_meas_mitigator_circuits(num_qubits, labels=labels)

[added]

The labels above are the $^7C_2$ possible choices of two qubit excitations and the $0^7$ and $1^7$ states.

In [57]:
print(meas_cal_circuits[-1])

        ┌───┐ ░ ┌─┐                  
   q_0: ┤ X ├─░─┤M├──────────────────
        ├───┤ ░ └╥┘┌─┐               
   q_1: ┤ X ├─░──╫─┤M├───────────────
        ├───┤ ░  ║ └╥┘┌─┐            
   q_2: ┤ X ├─░──╫──╫─┤M├────────────
        ├───┤ ░  ║  ║ └╥┘┌─┐         
   q_3: ┤ X ├─░──╫──╫──╫─┤M├─────────
        ├───┤ ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ┤ X ├─░──╫──╫──╫──╫─┤M├──────
        ├───┤ ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ┤ X ├─░──╫──╫──╫──╫──╫─┤M├───
        ├───┤ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   q_6: ┤ X ├─░──╫──╫──╫──╫──╫──╫─┤M├
        └───┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/═════════╩══╩══╩══╩══╩══╩══╩═
                 0  1  2  3  4  5  6 


[added]

`meas_cal_circuits` are the circuits corresponding to just the provided labels. However, `meas_cal_circuits_full` are  all possible $2^n$ circuits. `meas_cal_circuits` are used because for exp val computation, it suffices to take a subset of the set of all possible circuits.

In [17]:
[meas_cal_circuits_full, state_labels] = mit.complete_meas_cal(range(num_qubits))

In [63]:
print(meas_cal_circuits_full[4])

            ░ ┌─┐                  
q0_0: ──────░─┤M├──────────────────
            ░ └╥┘┌─┐               
q0_1: ──────░──╫─┤M├───────────────
      ┌───┐ ░  ║ └╥┘┌─┐            
q0_2: ┤ X ├─░──╫──╫─┤M├────────────
      └───┘ ░  ║  ║ └╥┘┌─┐         
q0_3: ──────░──╫──╫──╫─┤M├─────────
            ░  ║  ║  ║ └╥┘┌─┐      
q0_4: ──────░──╫──╫──╫──╫─┤M├──────
            ░  ║  ║  ║  ║ └╥┘┌─┐   
q0_5: ──────░──╫──╫──╫──╫──╫─┤M├───
            ░  ║  ║  ║  ║  ║ └╥┘┌─┐
q0_6: ──────░──╫──╫──╫──╫──╫──╫─┤M├
            ░  ║  ║  ║  ║  ║  ║ └╥┘
c0: 7/═════════╩══╩══╩══╩══╩══╩══╩═
               0  1  2  3  4  5  6 


## Run the circuits

We will run the circuits on the `ibmq_casablanca` quantum system.

In order to debug more quickly and avoid queues, you may consider using a simulator backend modeled after the real quantum system. This will use the noise model of `ibmq_casablanca` to do simulations. You may uncomment the line below to do so. **Note that the fidelities of your graph states will generally be higher on the simulator, but the Open Science Prize is awarded for the best fidelities on the real quantum system.**

In [27]:
# backend = QasmSimulator.from_backend(provider.get_backend('ibmq_casablanca'))
backend = provider.get_backend('ibmq_casablanca')

We run the measurement calibration circuits in a separate job from the graph state circuits.  We repeat both 16 times and use the mean as the final value. 

In order to debug more quickly, you may consider reducing `reps` from 16 to 1. **Note that the final submissions will need to be executed with 16 repetitions.**

In [45]:
reps = 6

In [46]:
all_jobs = []
all_jobs_mit = []

for ii in range(reps):

    # Run QPT on backend
    shots = 8192
    il = [0,1,2,3,4,5,6]
    
#     job_backend = execute(stabilizer_circuits, backend, shots=shots, initial_layout=il)
#     job_mit_backend = execute(meas_cal_circuits, backend, shots=shots, initial_layout=il)
    
    from qiskit.providers.ibmq.managed import IBMQJobManager
    from qiskit.compiler import transpile
    job_manager = IBMQJobManager()
    trans_stab_circs = transpile(stabilizer_circuits, backend=backend)
    trans_meas_cal_circs = transpile(meas_cal_circuits, backend=backend)
    job_backend = job_manager.run(trans_stab_circs, backend=backend)
    job_mit_backend = job_manager.run(trans_meas_cal_circs, backend=backend)
    
#     print('Job IDs ({}/{}): \n measurement calibration: {}\n stabilizer measurements: {}'.format(
#         ii+1, reps, job_mit_backend.job_id(), job_backend.job_id()))
    print('Job IDs ({}/{}): \n measurement calibration: {}\n stabilizer measurements: {}'.format(
        ii+1, reps, job_mit_backend.job_set_id(), job_backend.job_set_id()))

    all_jobs.append(job_backend)
    all_jobs_mit.append(job_mit_backend)

Job IDs (1/6): 
 measurement calibration: 67647c2255c04298b9efb8e45e87934f-16109674032059226
 stabilizer measurements: e389f1a584cf425a9231c0deec2c7dd6-16109673955966866
Job IDs (2/6): 
 measurement calibration: 0dcd4161b1b34b0ba96ee79d5a0d990a-16109674261616204
 stabilizer measurements: 37bb9d2a434c4ca48154804f23e17734-1610967418643612
Job IDs (3/6): 
 measurement calibration: 5c58d1af8ffa476d9cf06a4c78adb03d-16109674490517993
 stabilizer measurements: 99fe63be4baa4cefa15b710b0bac172b-161096744180763
Job IDs (4/6): 
 measurement calibration: 096c6a7bec514eea8425d03af65ff94d-16109674701948068
 stabilizer measurements: 33ca34f96e194a8ba5970610f0554154-16109674634362292
Job IDs (5/6): 
 measurement calibration: 82f36149d4c34c9d900257d194f9422b-16109674907107275
 stabilizer measurements: 7865bbd295004c03bc3e4bbafe4b10a1-16109674840172598
Job IDs (6/6): 
 measurement calibration: 3f5cbb840b884b4588742f3cd3c7cb4f-16109675118171215
 stabilizer measurements: 4eaeed7c4d5e47038273faffdd33f96c-1

We can monitor the status of the jobs using Qiskit's job monitoring tools.

In [ ]:
for job in all_jobs:
    job_monitor(job)
    try:
        if job.error_message() is not None:
            print(job.error_message())
    except:
        pass

## Post-processing and computing fidelities

Once the jobs are completed, we can get the results back as follows.

In [47]:
result_backend = []
result_mit_backend = []
for job in all_jobs:
    # Retrieve results (this may take a while depending on the queue)
#     result_backend.append(job.result())
    result_backend.append(job.results())
    
for job in all_jobs_mit:
#     result_mit_backend.append(job.result())
    result_mit_backend.append(job.results())

Finally, we compute the fidelities of the graph states. You may consider creating your own method for error mitigation by updating the `stabilizer_expvals` function below. Here, we will use the default methods provided in Qiskit.

In [48]:
def stabilizer_measure_diagonal(stabilizer):
    """Return the diagonal vector for a stabilizer measurement.
    
    Args:
        stabilizer (str): a stabilizer string
    
    Returns:
        np.ndarray: the diagonal for measurement in the stabilizer basis.
    """
    coeff, pauli = stabilizer_coeff_pauli(stabilizer)
    diag = np.array([1])
    for s in reversed(pauli):
        if s == 'I':
            tmp = np.array([1, 1])
        else:
            tmp = np.array([1, -1])
        diag = np.kron(tmp, diag)
    return coeff * diag
    
def stabilizer_fidelity(expvals, stddevs=None):
    """Compute stabilizer state fidelity from stabilizer expvals."""
    mean = np.mean(expvals)
    if stddevs is None:
        return mean
    stddev = np.sqrt(np.sum(stddevs ** 2))
    return mean, stddev

[added]

In the above cell, the function `stabilizer_fidelity()` simply just computes the mean of the expectation values (which is the total mean) and essentially computes the total standard deviation.

As for the function `stabilizer_measure_diagonal()`, if a stabilizer of the form $sP_1P_2\cdots P_k$ is given where $s$ is the sign, it returns the tensor $sQ_1\otimes Q_2 \otimes \cdots \otimes Q_k$ where $Q_i = \begin{bmatrix}1 & 1\end{bmatrix}$ if $P_i = I$ and $Q_i = \begin{bmatrix}1 & -1\end{bmatrix}$ if $P_i \neq I$. They call the tensor product as the diagonal vector for the given stabilizer.

### Your code goes here

You may consider updating the function below to change how the measurement calibration circuits are used to compute the fidelity of the graph state.

In [49]:
def stabilizer_expvals(result, stabilizers, meas_mitigator=None):
    """Compute expectation values from stabilizer measurement results."""

    ### YOUR CODE GOES HERE -- START
    
    expvals = []
    stddevs = []
    for i, stab in enumerate(stabilizers):
        expval, stddev = mit.expectation_value(
            result.get_counts(i),
            diagonal=stabilizer_measure_diagonal(stab),
            meas_mitigator=meas_mitigator)
        expvals.append(expval)
        stddevs.append(stddev)
    return np.array(expvals), np.array(stddevs)

    ### YOUR CODE GOES HERE -- END

In [50]:
## Mitigate the stabilizer expectation values 
F_nomit_backend = []
F_mit_backend = []

for ii in range(reps):
    # Unmitigated Expectation Values
    expvals_nomit_b, stddevs_nomit_b = stabilizer_expvals(
        result_backend[ii], stabilizers)
  
    # Fit measurement error mitigators
    mitigator_backend = mit.ExpvalMeasMitigatorFitter(result_mit_backend[ii], metadata).fit()

    # Measurement error mitigated expectation values
    expvals_mit_b, stddevs_mit_b = stabilizer_expvals(
        result_backend[ii], stabilizers, meas_mitigator=mitigator_backend)
    
    # save the fidelities for this iteration
    F_nomit_backend.append(stabilizer_fidelity(expvals_nomit_b, stddevs_nomit_b)[0])
    F_mit_backend.append(stabilizer_fidelity(expvals_mit_b, stddevs_mit_b)[0])

Report the fidelity estimates.

In [52]:
## The final results

print('Graph-state fidelity estimates')
print('\nNo mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_nomit_backend), np.std(F_nomit_backend)))

print('\nCTMP error mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_mit_backend), np.std(F_mit_backend)))

Graph-state fidelity estimates

No mitigation
F(ibmq_casablanca) = 0.592 ± 0.009

CTMP error mitigation
F(ibmq_casablanca) = 0.775 ± 0.018


## Qiskit version

In [21]:
import qiskit.tools.jupyter
%qiskit_version_table